#Model with Batch Normalization

Imports

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict

Prepare the data

In [ ]:
# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

Building the model

In [ ]:
loss_list=[]


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(OrderedDict([
            ('c1',nn.Conv2d(3,18,kernel_size=(5,5))),
            ('relu1',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2),stride=2)),
            ('c3',nn.Conv2d(18,48,kernel_size=(5,5))),
            ('relu3',nn.ReLU()),
            ('s4',nn.MaxPool2d(kernel_size=(2,2),stride=2)),
            ('c5',nn.Conv2d(48,360,kernel_size=(5,5))),
            ('batch_norm',nn.BatchNorm2d(360)),
            ('relu5',nn.ReLU())
 
            ]))
        
        self.fc = nn.Sequential(OrderedDict([
            ('f6',nn.Linear(360,84)),
            ('relu6',nn.ReLU()),
            ('f7',nn.Linear(84,10)),
            ('sig7',nn.LogSoftmax(dim=-1))
        ]))
        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        x_convnet = self.convnet(x)
        #print(x_convnet.size())
        x_convnet = x_convnet.view((-1,x_convnet.size()[1]))
        #print(x_convnet.size())
        x_fc = self.fc(x_convnet)
        out = x_fc
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out

Train function

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        #print(data.shape)
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.item())
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Test function

In [ ]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Main function

In [1]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = False
    save_model = True


    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)
    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    np.save('loss_list.npy',loss_list)
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..


100%|██████████| 170498071/170498071 [00:02<00:00, 68074631.96it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.305536
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.066761
Train Epoch: 1 [2560/50000 (5%)]	Loss: 1.897568
Train Epoch: 1 [3840/50000 (8%)]	Loss: 1.813623
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.755118
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.769610
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.688389
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.574329
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.778566
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.637904
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.644191
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.595572
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.671596
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.614861
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.741626
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.466430
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.557561
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.434208
Train Epoch: 1 [23040/5000